In [1]:
(require sicp)

# 构造过程抽象

## 程序设计基本元素

普遍性的认识  
- 将若干简单认识组合为一个复杂的认识，由此产生出各种复杂的认识
- 将2个认识放在一起对照，不管它们如何简单或者复杂，在这样做时并不将他们合二为一，由此可以得到有关它们的相互关系的认识。
- 将有关认识与那些在实际中和它们同在的所有其他认识隔离开，这就是抽象。

程序设计的基本元素  
- 基本表达形式 ： 用于表示语言所关心的最简单的个体。
- 组合的方法   ： 通过它们可以从比较简单的东西出发构造出复合的元素。
- 抽象的方法   ： 通过它们可以为复合对象命名，并将他们当作单元去操作。

组合式的求值  
- 求值该组合式的各个子表达式
- 将作为最左子表达式（运算符）的值的那个过程应用于相应的实际参数，所谓实际参数也就是其他子表达式（运算对象）的值。


### 抽象

这里以牛顿法求平方根来看看抽象，牛顿法是只需要求出y和x/y的平均值（它更接近实际的平均根值）

In [2]:
; 最顶层的抽象。
(define (sqrt-iter guess x)                       ; 定义
  (if (good-enough guess x)                       ; 如果这个值足够好了。
      guess                                       ; 就返回这个值就可以了。
      (sqrt-iter (improve guess x)                ; 否则继续迭代，然后improve是改进值。
                 x)))

; 如下是一步步的改进如上的表达式。
(define (improve guess x)                         ; 改进的
  (average guess (/ x  guess)))                   ; y和x/y的平均值

(define (good-enough guess x)                     ; 足够好了，误差足够小的意思
  (< (abs (- (square guess) x)) 0.001))  

(define (square x)                                ; 平方的
  (* x x))

(define (average x y)                             ; 平均值
(/ (+ x y) 2))

(sqrt-iter 1 9)

65537/21845

## 过程和它们所产生的运算 - 不就是时间复杂度吗

这里主要将递归和迭代的。  

- 我们说一个过程是递归的时候，论述的是语法形式上的事实，说明这个过程的定义中（直接或者间接的）引用了该过程自身。
- 在说某一计算过程具有某种模式时（例如线性递归），我们说的是这一计算过程的进展形式，而不是相应过程书写上的语法形式。

#### 斐波那契数列

In [5]:
; 例子是斐波那契数列，这一个序列中的每一个数都是前面2个数之和。

; 如下是递归版本
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else (+ (fib (- n 1))
                 (fib (- n 2))))))
(fib 8)

21

In [6]:
; 如下是迭代版本，迭代版本重要的是用几个变量来保存了中间的参数
; fib(1) = 1 , fib(0) = 0 , 然后反复的应用如下规则
; a <- a + b
; b <- a
(define (fib2 n)
  (define (fib2-iter a b counter)
    (if (= counter 0)
        b
        (fib2-iter (+ a b) a (- counter 1))))
  (fib2-iter 1 0 n))
(fib2 8)

21

可以看到两种方法所需的步骤相差巨大，递归版本的是一颗树，有很多重复的计算，而迭代版本的复杂性是线性的，

#### 换零钱

给了50、25、10、5、1美分的硬币，将1美元（100美分）换成零钱，总共有多少种换法？  

如下是递归法  

将总数为a的现金换成n种硬币的不同方式的数目等于
- 将现金数a换成除第一种硬币之外的所有其他硬币的不同方式数目，加上
- 将现金数a-d换成所有种类的硬币的不同方式数目，其中的d是第一种硬币的币值

按照如上的规则可以总结如下算法
- 如果a就是0，应该算作是有1种换零钱的方式。
- 如果a小于0，应该算作是有0种换零钱的方式。
- 如果n是0，应该算作是有0种换零钱的方式。

In [13]:
(define (first-denomination kinds-of-coin)
  (cond ((= kinds-of-coin 1) 1)
        ((= kinds-of-coin 2) 5)
        ((= kinds-of-coin 3) 10)
        ((= kinds-of-coin 4) 25)
        ((= kinds-of-coin 5) 50)))

(define (cc amount kinds-of-coin)
  (cond ((= amount 0) 1)
        ((or (< amount 0) (= kinds-of-coin 0)) 0)
        (else (+ (cc amount
                     (- kinds-of-coin 1))
                 (cc (- amount
                        (first-denomination kinds-of-coin))
                     kinds-of-coin)))))

(define (count-change amount)
  (cc amount 5))

(count-change 100)

292

In [28]:
; 如下是迭代法
(define (get-coin index)
  (cond ((= index 1) 50)
        ((= index 2) 25)
        ((= index 3) 10)
        ((= index 4) 5)
        ((= index 5) 1)))

(define (c-c leftAmount count cursor c1 c2 c3 c4 c5)
  (cond ((or (= leftAmount 0) (< leftAmount 0)) ; 当剩余的钱小于或等于0的时候
         (cond ((or (= cursor 4)
                    (and (= cursor 5) (> c4 0))) ;如果cursor=4，或者 cursor=5且c4>0
                (c-c (- (+ leftAmount
                           (get-coin 4))
                        (get-coin 5)) ;leftAmount将第4种硬币钱数加回来一个，且c4减1，紧接着加上第5种硬币钱数，且c5加1
                     (if (< leftAmount 0)
                         count
                         (+ count 1)) ;如果leftAmount=0，说明正好分完一次，所以count加1；反之，count不变
                     5 ;将游标指向第五种硬币
                     c1
                     c2
                     c3
                     (- c4 1) ;c4减1
                     (+ c5 1))) ;c5加1，不清零是因为c5是最后一种硬币，无需从零开始计算
                ((or (= cursor 3)
                     (and (= cursor 5) (= c4 0) (> c3 0)))
                 (c-c (- (+ leftAmount
                            (get-coin 3)
                            (* c5 (get-coin 5)));此时c5需要清零，所以把第五种硬币的钱数都加回来
                          (get-coin 4))
                      (if (< leftAmount 0)
                          count
                          (+ count 1))
                      4
                      c1
                      c2
                      (- c3 1)
                      (+ c4 1)
                      0))
               ((or (= cursor 2)
                    (and (= cursor 5) (= c4 0) (= c3 0) (> c2 0)))
                (c-c (- (+ leftAmount
                           (get-coin 2)
                           (* c4 (get-coin 4))
                           (* c5 (get-coin 5)))
                        (get-coin 3))
                     (if (< leftAmount 0)
                         count
                         (+ count 1))
                     3
                     c1
                     (- c2 1)
                     (+ c3 1)
                     0 ;清零原理同上
                     0))
               ((or (= cursor 1)
                    (and (= cursor 5) (= c4 0) (= c3 0) (= c2 0) (> c1 0)))
                (c-c (- (+ leftAmount
                           (get-coin 1)
                           (* c3 (get-coin 3))
                           (* c4 (get-coin 4))
                           (* c5 (get-coin 5)))
                        (get-coin 2))
                     (if (< leftAmount 0)
                         count
                         (+ count 1))
                     2
                     (- c1 1)
                     (+ c2 1)
                     0
                     0
                     0))
               (else (if (< leftAmount 0); 此时c4=0,c3=0,c2=0,c1=0,所以全部遍历完毕，结束。
                         count
                         (+ count 1)))))
         (else (c-c (- leftAmount
                       (get-coin cursor));此时如果leftAmount>0,继续将游标指向的硬币数量加1
                    count
                    cursor
                    (if (= cursor 1)
                        (+ c1 1)
                        c1)
                    (if (= cursor 2)
                        (+ c2 1)
                        c2)
                    (if (= cursor 3)
                        (+ c3 1)
                        c3)
                    (if (= cursor 4)
                        (+ c4 1)
                        c4)
                    (if (= cursor 5)
                        (+ c5 1)
                        c5)))))

(define (count-change2 amount)
  (c-c (- amount
          (get-coin 1))
       0
       1
       1
       0
       0
       0
       0))
(count-change2 100)

292

总结：  
- 递归 ： 程序调用自己，递归通常是从顶部将问题分解，通过解决掉所有分解出来的小问题，来解决整个问题。
- 迭代 ： 利用变量的原值推算出变量的新值，递归中肯定有迭代，但迭代中不一定有递归。
- 动态规划 ： 通常于递归相反，其从底部开始解决问题，将所有小问题解决掉，进而解决了整个问题。

## 用高阶函数做抽象

In [32]:
; 首先考虑如下的几个函数，都是求和的
; 比如这个是计算从a到b的各个整数之和
(define (sum-integers a b)
  ( if (> a b)                         ; a不断递增，有界限
    0                                  
    (+ a (sum-integers (+ a 1) b))))   ; 不断的加

; 这里求给定范围内整数的立方之和
(define (cube x) (* x x x))

(define (sum-cube a b)
  (if (> a b)
      0
      (+ (cube a) (sum-cube (+ a 1) b))))

; 还有很多其他的求和，可以看到共享着一种公共的基础模式
(define (mysum term a next b)
  (if (> a b)
      0 
      (+ (term a)                          ; 这里term是对这个整数计算
         (mysum term (next a) next b))))   ; next表示下一个。
; 所以这里求a到b的各个整数立方之和就可以这样做
(define (inc n) (+ n 1))  ; next模式是递增
(define (sum-cube a b)
  (mysum cube a inc b))

(sum-cube 1 2)



9

# 引用
> 